<a href="https://colab.research.google.com/github/C23-PS286-Capstone-Project/skinsight-machine-learning/blob/main/Notebook/SkinSight_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Library For ML**

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import humanize
import random
import re
import shutil
import math
import multiprocessing
import asyncio
import concurrent.futures

from google.colab import files
from PIL import Image
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report